In [1]:
import rospy
import time
import jupyros as jr
import matplotlib
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import math
import actionlib
import actionlib.msg

%matplotlib widget

from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from std_msgs.msg import String, Float64, Bool, Float32
from rt2_assignment1.srv import Command
from ipywidgets import interact,interactive,fixed,interact_manual
from geometry_msgs.msg import Twist

In [2]:
rospy.init_node('user_interface')
client=rospy.ServiceProxy('/user_interface', Command)
Vel_Publisher = rospy.Publisher('/vel', Twist, queue_size = 1)
Command_Publisher = rospy.Publisher('/cmd_vel', Twist, queue_size = 1)
velocity = Twist()
speed = Twist()

In [3]:
def Random_start(r):
    global random_behaviour
    random_behaviour = True
    client('start')

def Random_stop(r):
    global random_behaviour
    random_behaviour = False
    client('stop')

def Onwards(t):
    global random_behaviour, speed, velocity
    if random_behaviour:
        random_behaviour = False
        client('stop')
    Stop(1)
    velocity.linear.x = speed.linear.x

    Command_Publisher.publish(velocity)
    
def Backwards(t):
    global random_behaviour, speed, velocity
    if random_behaviour:
        random_behaviour = False
        client('stop')
    Stop(1)
    velocity.linear.x = - speed.linear.x

    Command_Publisher.publish(velocity)

def leftwards(t):
    global random_behaviour, speed, velocity
    if random_behaviour:
        random_behaviour = False
        client('stop')
    Stop(1)
    velocity.angular.z = - speed.angular.z

    Command_Publisher.publish(velocity)

def rightwards(t):
    global random_behaviour, speed, velocity
    if random_behaviour:
        random_behaviour = False
        client('stop')
    Stop(1)
    velocity.angular.z = speed.angular.z

    Command_Publisher.publish(velocity)

def Stop(s):
    velocity.linear.x = 0
    velocity.angular.z = 0
    Command_Publisher(velocity)

def Set_Linear_Velocity(vel):
    global speed
    speed.linear.x = vel
    Vel_Publisher.publish(speed)
    return vel

def Set_Angular_Velocity(vel):
    global speed
    speed.angular.z = vel
    Vel_Publisher.publish(speed)
    return vel

In [4]:
Button_rand_go=widgets.Button(
    description='Start Random Behaviour',
    style=widgets.ButtonStyle(button_color='blue'),
    layout=widgets.Layout(width='auto', grid_area='but_rand_go')
)

Button_rand_stop=widgets.Button(
    description='Stop Random Behaviour',
    style=widgets.ButtonStyle(button_color='purple'),
    layout=widgets.Layout(width='auto', grid_area='but_rand_stop')
)

Button_onwards=widgets.Button(
    description='Onwards',
    style=widgets.ButtonStyle(button_color='pink'),
    layout=widgets.Layout(width='auto', grid_area='but_onwards')
)

Button_backwards=widgets.Button(
    description='Backwards',
    style=widgets.ButtonStyle(button_color='pink'),
    layout=widgets.Layout(width='auto', grid_area='but_backwards')
)

Button_Leftwards=widgets.Button(
    description='Left',
    style=widgets.ButtonStyle(button_color='pink'),
    layout=widgets.Layout(width='auto', grid_area='but_left')
)

Button_Rightwards=widgets.Button(
    description='Right',
    style=widgets.ButtonStyle(button_color='pink'),
    layout=widgets.Layout(width='auto', grid_area='but_right')
)

Button_stop=widgets.Button(
    description='Stop',
    style=widgets.ButtonStyle(button_color='red'),
    layout=widgets.Layout(width='auto', grid_area='but_stop')
)


In [5]:
Button_rand_go.on_click(Random_start)
Button_rand_stop.on_click(Random_stop)
Button_onwards.on_click(Onwards)
Button_backwards.on_click(Backwards)
Button_Leftwards.on_click(leftwards)
Button_Rightwards.on_click(rightwards)
Button_stop.on_click(Stop)

In [6]:
print("Linear Velocity Value")
interact(Set_Linear_Velocity, vel =widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5))

print("Angular Velocity Value")
interact(Set_Angular_Velocity, vel =widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5))

widgets.GridBox(children=[Button_rand_go, Button_rand_stop, Button_onwards, Button_backwards, Button_Leftwards, Button_Rightwards, Button_stop],
                layout=widgets.Layout(width='80%',grid_template_rows='auto auto',
                grid_template_columns='32% 16% 16% 16% 16% ',
                grid_template_areas='''"but_rand_go . . but_onwards but_stop ""but_rand_stop . but_left but_backwards but_right "''')
               )

Linear Velocity Value


interactive(children=(FloatSlider(value=0.5, description='vel', max=1.0, step=0.01), Output()), _dom_classes=(…

Angular Velocity Value


interactive(children=(FloatSlider(value=0.5, description='vel', max=1.0, step=0.01), Output()), _dom_classes=(…

GridBox(children=(Button(description='Start Random Behaviour', layout=Layout(grid_area='but_rand_go', width='a…

...

TypeError: 'Publisher' object is not callable

TypeError: 'Publisher' object is not callable

TypeError: 'Publisher' object is not callable

TypeError: 'Publisher' object is not callable

TypeError: 'Publisher' object is not callable